In [360]:
import pandas as pd
from datetime import datetime

In [12]:
import tensorflow as tf
import keras
import sklearn
import cv2 
import torch

In [438]:
data = pd.read_csv("/Users/luisfelipemh/Desktop/Personal/pruebas_seleccion/tissini/Data 2.csv",delimiter=",",encoding = 'unicode_escape')

In [439]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [499]:
#saber el producto más vendido
data.groupby(['Description']).sum()['Quantity'].sort_values(ascending=False)

Description
PAPER CRAFT , LITTLE BIRDIE           161990
MEDIUM CERAMIC TOP STORAGE JAR        152527
WORLD WAR 2 GLIDERS ASSTD DESIGNS      56247
JUMBO BAG RED RETROSPOT                49593
WHITE HANGING HEART T-LIGHT HOLDER     40473
                                       ...  
PINK BERTIE MOBILE PHONE CHARM             1
PINK CRYSTAL GUITAR PHONE CHARM            1
PINK FEATHER CHRISTMAS DECORATION          1
PINK LARGE JEWELED PHOTOFRAME              1
FLOWER SHOP DESIGN MUG                     1
Name: Quantity, Length: 4223, dtype: int64

In [440]:
data[data['CustomerID']==12346.0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/2011 10:01,1.04,12346.0,United Kingdom
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,1/18/2011 10:17,1.04,12346.0,United Kingdom


In [441]:
for i in data.columns:
    print(i,data[i].dtype)

InvoiceNo object
StockCode object
Description object
Quantity int64
InvoiceDate object
UnitPrice float64
CustomerID float64
Country object


In [442]:
data['CustomerID']=data['CustomerID'].astype(object)

In [443]:
for i in data.columns:
    print(i,data[i].dtype)

InvoiceNo object
StockCode object
Description object
Quantity int64
InvoiceDate object
UnitPrice float64
CustomerID object
Country object


In [444]:
data['Quantity']=data['Quantity'].abs()
data['value']=data['Quantity']*data['UnitPrice']

In [445]:
def cancellation(x):
    if "C" in str(x):
        return "C"
    else:
        return "N"

data['cancellation']=data['InvoiceNo'].apply(lambda x: cancellation(x))

data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,value,cancellation
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,N
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,N
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,N
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,N
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,N


In [446]:
data[data['cancellation']=='C']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,value,cancellation
141,C536379,D,Discount,1,12/1/2010 9:41,27.50,14527.0,United Kingdom,27.50,C
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,1,12/1/2010 9:49,4.65,15311.0,United Kingdom,4.65,C
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,12,12/1/2010 10:24,1.65,17548.0,United Kingdom,19.80,C
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,24,12/1/2010 10:24,0.29,17548.0,United Kingdom,6.96,C
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,24,12/1/2010 10:24,0.29,17548.0,United Kingdom,6.96,C
...,...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,11,12/9/2011 9:57,0.83,14397.0,United Kingdom,9.13,C
541541,C581499,M,Manual,1,12/9/2011 10:28,224.69,15498.0,United Kingdom,224.69,C
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,5,12/9/2011 11:57,10.95,15311.0,United Kingdom,54.75,C
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,1,12/9/2011 11:58,1.25,17315.0,United Kingdom,1.25,C


In [447]:
data_customer=pd.pivot_table(data,index='CustomerID',columns=['cancellation','Description'],values='Quantity',aggfunc='sum').fillna(0)

In [448]:
data_customer.columns = data_customer.columns.map(' - '.join).str.strip(' - ')

In [449]:
data_customer=data_customer.reset_index().rename(columns={'CustomerID':'ID'}).set_index('ID')

In [450]:
columns_c=[i for i in data_customer.columns if i[0]=='C']
columns_nc=[i for i in data_customer.columns if i[0]=='N']

In [451]:
data_customer.filter(items=[12346.0],axis=0)[['C - MEDIUM CERAMIC TOP STORAGE JAR','N - MEDIUM CERAMIC TOP STORAGE JAR']]

,C - MEDIUM CERAMIC TOP STORAGE JAR,N - MEDIUM CERAMIC TOP STORAGE JAR
ID,,
12346.0,74215.0,74215.0


In [452]:
data_customer[columns_c].sum(axis=1)

ID
12346.0    74215.0
12347.0        0.0
12348.0        0.0
12349.0        0.0
12350.0        0.0
            ...   
18280.0        0.0
18281.0        0.0
18282.0        5.0
18283.0        0.0
18287.0        0.0
Length: 4372, dtype: float64

In [453]:
data_customer[columns_nc].sum(axis=1)

ID
12346.0    74215.0
12347.0     2470.0
12348.0     2341.0
12349.0      631.0
12350.0      209.0
            ...   
18280.0       45.0
18281.0       54.0
18282.0      103.0
18283.0     1398.0
18287.0     1586.0
Length: 4372, dtype: float64

In [454]:
data_customer=data_customer.join(data_customer[columns_c].sum(axis=1).rename('quantity_c')).join(data_customer[columns_nc].sum(axis=1).rename('quantity_nc'))

In [455]:
data_customer

,C - 50'S CHRISTMAS GIFT BAG LARGE,C - DOLLY GIRL BEAKER,C - OVAL WALL MIRROR DIAMANTE,C - SET 2 TEA TOWELS I LOVE LONDON,C - SPACEBOY BABY GIFT SET,C - TRELLIS COAT RACK,C - 10 COLOUR SPACEBOY PEN,C - 12 COLOURED PARTY BALLOONS,C - 12 EGG HOUSE PAINTED WOOD,C - 12 IVORY ROSE PEG PLACE SETTINGS,...,N - ZINC SWEETHEART WIRE LETTER RACK,N - ZINC T-LIGHT HOLDER STAR LARGE,N - ZINC T-LIGHT HOLDER STARS LARGE,N - ZINC T-LIGHT HOLDER STARS SMALL,N - ZINC TOP 2 DOOR WOODEN SHELF,N - ZINC WILLIE WINKIE CANDLE STICK,N - ZINC WIRE KITCHEN ORGANISER,N - ZINC WIRE SWEETHEART LETTER TRAY,quantity_c,quantity_nc
ID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74215.0,74215.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2470.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2341.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,631.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0
18281.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0
18282.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,103.0


In [456]:
data_customer['percentage_c']=data_customer['quantity_c']/(data_customer['quantity_c']+data_customer['quantity_nc'])

In [457]:
data_customer[data_customer['percentage_c']>=0.5]

,C - 50'S CHRISTMAS GIFT BAG LARGE,C - DOLLY GIRL BEAKER,C - OVAL WALL MIRROR DIAMANTE,C - SET 2 TEA TOWELS I LOVE LONDON,C - SPACEBOY BABY GIFT SET,C - TRELLIS COAT RACK,C - 10 COLOUR SPACEBOY PEN,C - 12 COLOURED PARTY BALLOONS,C - 12 EGG HOUSE PAINTED WOOD,C - 12 IVORY ROSE PEG PLACE SETTINGS,...,N - ZINC T-LIGHT HOLDER STAR LARGE,N - ZINC T-LIGHT HOLDER STARS LARGE,N - ZINC T-LIGHT HOLDER STARS SMALL,N - ZINC TOP 2 DOOR WOODEN SHELF,N - ZINC WILLIE WINKIE CANDLE STICK,N - ZINC WIRE KITCHEN ORGANISER,N - ZINC WIRE SWEETHEART LETTER TRAY,quantity_c,quantity_nc,percentage_c
ID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74215.0,74215.0,0.500000
12454.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1006.0,1006.0,0.500000
12503.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000
12505.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000
12558.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,196.0,0.500000
12605.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.000000
12607.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1228.0,1228.0,0.500000
12666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,1.000000
12870.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.000000


In [458]:
data['InvoiceDate']

0          12/1/2010 8:26
1          12/1/2010 8:26
2          12/1/2010 8:26
3          12/1/2010 8:26
4          12/1/2010 8:26
               ...       
541904    12/9/2011 12:50
541905    12/9/2011 12:50
541906    12/9/2011 12:50
541907    12/9/2011 12:50
541908    12/9/2011 12:50
Name: InvoiceDate, Length: 541909, dtype: object

In [459]:
def str_to_date(x):
    x=datetime.strptime(x,"%d/%m/%Y %H:%M")
    return x

In [460]:
def transformador_hora(x):
    x = pd.to_datetime(x).strftime('%d/%m/%Y %H:%M')
    return x

In [461]:
data['Fecha']=data['InvoiceDate'].apply(lambda x: transformador_hora(x))

In [462]:
data['Fecha']

0         01/12/2010 08:26
1         01/12/2010 08:26
2         01/12/2010 08:26
3         01/12/2010 08:26
4         01/12/2010 08:26
                ...       
541904    09/12/2011 12:50
541905    09/12/2011 12:50
541906    09/12/2011 12:50
541907    09/12/2011 12:50
541908    09/12/2011 12:50
Name: Fecha, Length: 541909, dtype: object

In [463]:
data['Fecha']=data['Fecha'].apply(lambda x: str_to_date(x))

In [464]:
data['Fecha']

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: Fecha, Length: 541909, dtype: datetime64[ns]

In [465]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,value,cancellation,Fecha
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,N,2010-12-01 08:26:00
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,N,2010-12-01 08:26:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,N,2010-12-01 08:26:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,N,2010-12-01 08:26:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,N,2010-12-01 08:26:00
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20,N,2011-12-09 12:50:00
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60,N,2011-12-09 12:50:00
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60,N,2011-12-09 12:50:00
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60,N,2011-12-09 12:50:00


In [482]:
time_series=data[data['cancellation']=='N'][['Fecha','value','Quantity','Country']]

In [483]:
month=pd.pivot_table(time_series,index=[time_series['Fecha'].dt.year,time_series['Fecha'].dt.month],values='Quantity',aggfunc='sum').droplevel(level=0).reset_index().rename(columns={'Fecha':'Mes'})

In [484]:
year=pd.pivot_table(time_series,index=[time_series['Fecha'].dt.year,time_series['Fecha'].dt.month],values='Quantity',aggfunc='sum').droplevel(level=1).reset_index().rename(columns={'Fecha':'Ano'})

In [485]:
time_series=year.join(month['Mes'])

In [486]:
time_series.dtypes

Ano         int64
Quantity    int64
Mes         int64
dtype: object

In [487]:
time_series['Ano']=time_series['Ano'].astype(str)
time_series['Mes']=time_series['Mes'].astype(str)

In [488]:
time_series.dtypes

Ano         object
Quantity     int64
Mes         object
dtype: object

In [489]:
time_series['Fecha']=time_series['Ano']+"/"+time_series['Mes']

In [490]:
pd.to_datetime(time_series['Fecha'])

0    2010-12-01
1    2011-01-01
2    2011-02-01
3    2011-03-01
4    2011-04-01
5    2011-05-01
6    2011-06-01
7    2011-07-01
8    2011-08-01
9    2011-09-01
10   2011-10-01
11   2011-11-01
12   2011-12-01
Name: Fecha, dtype: datetime64[ns]

In [491]:
time_series.drop(['Ano','Mes'],axis=1)

,Quantity,Fecha
0,366362,2010/12
1,406621,2011/1
2,292530,2011/2
3,412633,2011/3
4,321609,2011/4
5,412591,2011/5
6,439689,2011/6
7,417934,2011/7
8,431487,2011/8
9,593586,2011/9


In [492]:
time_series['Fecha']=pd.to_datetime(time_series['Fecha'])

In [493]:
time_series.drop(['Ano','Mes'],axis=1).set_index('Fecha')

,Quantity
Fecha,
2010-12-01,366362
2011-01-01,406621
2011-02-01,292530
2011-03-01,412633
2011-04-01,321609
2011-05-01,412591
2011-06-01,439689
2011-07-01,417934
2011-08-01,431487
